### Load the prerequisites

In [1]:
import numpy as np
import pandas as pd
eps = np.finfo(float).eps
from numpy import log2 as log

We will read this with Pandas dataframe

In [2]:
df = pd.read_csv("id3.csv")

In [4]:
def find_entropy(df):
    Class = df.keys()[-1]   #To make the code generic, changing target variable class name
    entropy = 0
    values = df[Class].unique()
    for value in values:
        fraction = df[Class].value_counts()[value]/len(df[Class])
        entropy += -fraction*np.log2(fraction)
    return entropy
  
  
def find_entropy_attribute(df,attribute):
  Class = df.keys()[-1]   #To make the code generic, changing target variable class name
  target_variables = df[Class].unique()  #This gives all 'Yes' and 'No'
  variables = df[attribute].unique()    #This gives different features in that attribute (like 'Hot','Cold' in Temperature)
  entropy2 = 0
  for variable in variables:
      entropy = 0
      for target_variable in target_variables:
          num = len(df[attribute][df[attribute]==variable][df[Class] ==target_variable])
          den = len(df[attribute][df[attribute]==variable])
          fraction = num/(den+eps)
          entropy += -fraction*log(fraction+eps)
      fraction2 = den/len(df)
      entropy2 += -fraction2*entropy
  return abs(entropy2)


def find_winner(df):
    Entropy_att = []
    IG = []
    for key in df.keys()[:-1]:
#         Entropy_att.append(find_entropy_attribute(df,key))
        IG.append(find_entropy(df)-find_entropy_attribute(df,key))
    return df.keys()[:-1][np.argmax(IG)]
  
  
def get_subtable(df, node,value):
  return df[df[node] == value].reset_index(drop=True)


def buildTree(df,tree=None): 
    Class = df.keys()[-1]   #To make the code generic, changing target variable class name
    
    #Here we build our decision tree

    #Get attribute with maximum information gain
    node = find_winner(df)
    
    #Get distinct value of that attribute e.g Salary is node and Low,Med and High are values
    attValue = np.unique(df[node])
    
    #Create an empty dictionary to create tree    
    if tree is None:                    
        tree={}
        tree[node] = {}
    
   #We make loop to construct a tree by calling this function recursively. 
    #In this we check if the subset is pure and stops if it is pure. 

    for value in attValue:
        
        subtable = get_subtable(df,node,value)
        clValue,counts = np.unique(subtable['Answer'],return_counts=True)                        
        
        if len(counts)==1:#Checking purity of subset
            tree[node][value] = clValue[0]                                                    
        else:        
            tree[node][value] = buildTree(subtable) #Calling the function recursively 
                   
    return tree

Now we call the buildTree function and print the tree we built.

In [5]:
tree = buildTree(df)

In [6]:
import pprint
#pprint.pprint(tree)

{'Outlook': {'overcast': 'yes',
             'rain': {'Wind': {'strong': 'no', 'weak': 'yes'}},
             'sunny': {'Humidity': {'high': 'no', 'normal': 'yes'}}}}


You can see the tree structure is formed based on the priority list to split the data.

We can write an algorithm to predict using this tree structure.

In [7]:
def predict(inst,tree):
    #This function is used to predict for any input variable 
    
    #Recursively we go through the tree that we built earlier

    for nodes in tree.keys():        
        
        value = inst[nodes]
        tree = tree[nodes][value]
        prediction = 0
            
        if type(tree) is dict:
            prediction = predict(inst, tree)
        else:
            prediction = tree
            break;                            
        
    return prediction

### Prediction

Get prediction

In [14]:
data = {'Outlook':'overcast', 'Temperature':'cool', 'Humidity':'normal', 'Wind': 'strong'}

In [15]:
inst = pd.Series(data)

In [20]:
print("Defined decision tree: \n")
pprint.pprint(tree)
count = 0
total = 10
print("\nPredicting...\nData - Given - Predicted")
for i in range(4, 14):
    inst = df.iloc[i]
    y = inst['Answer']
    x = inst[df.columns.values[:-1]]
    y_hat = predict(x, tree)
    
    print("{0} - {1} - {2}".format(str(inst[df.columns.values[:-1]].to_dict()), y, y_hat))
    
    if y==y_hat:
        count+=1
acc = count/total
print("Accuracy: {0}".format(str(acc)))
    
    
    

Defined decision tree: 

{'Outlook': {'overcast': 'yes',
             'rain': {'Wind': {'strong': 'no', 'weak': 'yes'}},
             'sunny': {'Humidity': {'high': 'no', 'normal': 'yes'}}}}

Predicting...
Data - Given - Predicted
{'Outlook': 'rain', 'Temperature': 'cool', 'Humidity': 'normal', 'Wind': 'weak'} - yes - yes
{'Outlook': 'rain', 'Temperature': 'cool', 'Humidity': 'normal', 'Wind': 'strong'} - no - no
{'Outlook': 'overcast', 'Temperature': 'cool', 'Humidity': 'normal', 'Wind': 'strong'} - yes - yes
{'Outlook': 'sunny', 'Temperature': 'mild', 'Humidity': 'high', 'Wind': 'weak'} - no - no
{'Outlook': 'sunny', 'Temperature': 'cool', 'Humidity': 'normal', 'Wind': 'weak'} - yes - yes
{'Outlook': 'rain', 'Temperature': 'mild', 'Humidity': 'normal', 'Wind': 'weak'} - yes - yes
{'Outlook': 'sunny', 'Temperature': 'mild', 'Humidity': 'normal', 'Wind': 'strong'} - yes - yes
{'Outlook': 'overcast', 'Temperature': 'mild', 'Humidity': 'high', 'Wind': 'strong'} - yes - yes
{'Outlook': 'o